In [1]:
######## # AUTO LOGIN-1 #########
account_name = 'haj'
import pandas as pd
import numpy as np
login_str = 'login_'+account_name
import threading
import importlib
login_str = 'login_'+account_name
module = importlib.import_module(login_str, package=None)
print('Logging in with account : '+str(account_name))
module.retry_autologin()

Logging in with account : haj
Trying to login...
access token : pPliErEgT1EK5IwSoJOziQePnGqx2eh3
request token : On5EJ3ZOhEGm5dOpQ8s2x4cghxBZFIO1


In [2]:
my_qty = 0
qty_m_spot = 200

ma = 15
srnum = 24
qty_factor = 15
threshold = 35
ex_type = module.kite.EXCHANGE_CDS
inst_token = 299011

,buy_quantity,sell_quantity,diff,mean,percent,percent_10ma,percent_diff,percent_diff_10ma,trend,my_qty,ltp,date
0,209927,163119,46808,186523.0,25.095028,25.095028,0.000000,0.000000,0,0,74.8150,2021-04-09 10:13:44
1,234005,176231,57774,205118.0,28.166226,28.166226,0.000000,0.000000,0,0,74.9450,2021-04-09 11:35:34
2,234006,176312,57694,205159.0,28.121603,28.143915,-0.079277,-0.039638,0,0,74.9450,2021-04-09 11:35:34
3,234030,176367,57663,205198.5,28.101083,28.129637,-0.101511,-0.060263,0,0,74.9450,2021-04-09 11:35:35
4,233981,176377,57604,205179.0,28.074998,28.115978,-0.145752,-0.081635,0,0,74.9475,2021-04-09 11:35:36
...,...,...,...,...,...,...,...,...,...,...,...,...
6048,239623,165775,73848,202699.0,36.432345,37.142935,-1.913121,-0.825230,0,-220,74.8925,2021-04-09 12:58:55
6049,239623,165775,73848,202699.0,36.432345,37.019722,-1.586658,-0.960154,0,-220,74.8925,2021-04-09 12:58:55
6050,240445,166921,73524,203683.0,36.097269,36.854495,-2.054638,-1.187558,0,-220,74.8925,2021-04-09 12:58:56
6051,240513,166791,73722,203652.0,36.199988,36.699541,-1.361196,-1.346680,0,-220,74.8975,2021-04-09 12:58:57


In [ ]:
depth_data_old = pd.read_csv('./depth_data_8_1.csv')
del depth_data_old['Unnamed: 0']
my_qty = depth_data_old['my_qty'][depth_data_old.index.values[-1]]
depth_data_old

In [3]:
class PnLCalculator:
    def __init__(self):
        self.quantity = 0
        self.cost = 0.0
        self.market_value = 0.0
        self.r_pnl = 0.0
        self.average_price = 0.0
        self.date = 0.0

    def fill(self, n_pos, exec_price,date):
        pos_change = n_pos - self.quantity
        direction = np.sign(pos_change)
        prev_direction = np.sign(self.quantity)
        qty_closing = min(abs(self.quantity), abs(pos_change)) * direction if prev_direction != direction else 0
        qty_opening = pos_change if prev_direction == direction else pos_change - qty_closing

        new_cost = self.cost + qty_opening * exec_price
        if self.quantity != 0:
            new_cost += qty_closing * self.cost / self.quantity
            self.r_pnl += qty_closing * (self.cost / self.quantity - exec_price)

        self.quantity = n_pos
        self.cost = new_cost
        self.date = date

    def update(self, price):
        if self.quantity != 0:
            self.average_price = self.cost / self.quantity
        else: 
            self.average_price = 0
        self.market_value = self.quantity * price
        return self.market_value - self.cost

In [4]:
def export_calc():
    npdsf = pd.DataFrame(log_data_lst)
    npdsf = pd.concat([depth_data_old,npdsf])
    ndf =  npdsf[npdsf.trend != 0]
    cnt1 = 0
    positions_call = ndf['my_qty']
    exec_prices_call = ndf['ltp']
    bkg_and_charges = 64.5
    date_call = ndf['date']
    pnls_call = []
    pos_call = PnLCalculator()
    for (p,e,dt) in zip(positions_call, exec_prices_call,date_call):
        pos_call.fill(p, e,dt)
        u_pnl = pos_call.update(e)
        pnls_call.append({'date':pos_call.date,
                          'Qty':int(pos_call.quantity),
                          'r_pnl':round((1000*pos_call.r_pnl),4),
                          'u_pnl':round(1000*u_pnl,4),
                          'total_pnl':round(1000*(pos_call.r_pnl+u_pnl)-(cnt1*(bkg_and_charges/2)),4),
                          'average_price':round(pos_call.average_price,4),
                         })
        cnt1 = cnt1+1

    pnls_df_call = pd.DataFrame(pnls_call)
    # print(pnls_df_call)
    # pnls_df_call['total_pnl'].plot()
    npdsf.to_csv('./depth_data_8_1.csv')
    pnls_df_call.to_csv('./pnls_df_call_4_1.csv')

In [ ]:
# base functions : getquant, placeneworder, ckqnt

# myquantity = 0
# def getquant():
#     global myquantity,order_type,symbol_ip
#     allpos = kite.positions()['net']
#     for i in range(len(allpos)):
#         if(allpos[i]['tradingsymbol']==symbol_ip and  allpos[i]['product']== order_type):
#             myquantity = allpos[i]['quantity']
#             print('My Quantity : ' + str(allpos[i]['quantity']))

stop_trading = False
stopbuy = False
stopsell = False


def placeneworder(quantdiff,price_ip):
    '''placeneworder(quantdiff,price_ip)
    here price_ip =0 because market order
    '''
    global stopbuy,stop_trading,stopsell,order_type,symbol_ip,ex_type
    if(quantdiff>0 and stopbuy == False and stop_trading==False):
        try:
            stop_trading = True
            order_id= module.kite.place_order(tradingsymbol=symbol_ip,
                        exchange=ex_type,
                        transaction_type=module.kite.TRANSACTION_TYPE_BUY,
                        quantity=abs(quantdiff),
#                             price=price_ip,
                        order_type=module.kite.ORDER_TYPE_MARKET,
                        variety = module.kite.VARIETY_REGULAR,
                        product=module.kite.PRODUCT_MIS if order_type=='MIS' else module.kite.PRODUCT_NRML)
            getquant()
#             stopsell = False
#             stop_trading = False
        except Exception as e:
#             stopbuy = True
            print(e)
            getquant()
    if(quantdiff<0 and stopsell == False and stop_trading==False):
        try:
            stop_trading = True
            order_id= module.kite.place_order(tradingsymbol=symbol_ip,
                        exchange=ex_type,
                        transaction_type=module.kite.TRANSACTION_TYPE_SELL,
                        quantity=abs(quantdiff),
#                             price=price_ip,
                        order_type=module.kite.ORDER_TYPE_MARKET,
                        variety = module.kite.VARIETY_REGULAR,
                        product=module.kite.PRODUCT_MIS if order_type=='MIS' else module.kite.PRODUCT_NRML)
            getquant()
#             stopbuy = False
#             stop_trading = False
        except Exception as e:
#             stopsell = True
            print(e)
            getquant()
        
# def ckqnt (orderprice,orderquant):
#     global myquantity
#     quantdiff = orderquant - myquantity
#     placeneworder(quantdiff,orderprice)
#     return myquantity


def Average(lst): 
    return sum(lst) / len(lst) 


In [ ]:
buy_qty_lst = []
sell_qty_lst = []
percent_lst = []
percent_diff_lst = []
log_data_lst = []

In [ ]:
import logging
from kiteconnect import KiteTicker
logging.basicConfig(level=logging.DEBUG)

counter = 0

def on_ticks(ws, ticks):
    global my_qty,counter
  
    print('==========================')
    buy_quantity = ticks[0]['buy_quantity']
    sell_quantity = ticks[0]['sell_quantity']
    diff = buy_quantity - sell_quantity
    mean = (buy_quantity + sell_quantity)/2
    percent  = (diff/mean)*100
    percent_lst.append(percent)
    if(len(percent_lst)>ma):
        percent_lst.pop(0)
    
    percent_10ma = Average(percent_lst)
    percent_diff = 100*(percent-percent_10ma)/percent_10ma
    print('percent_diff : '+str(round(percent_diff,2)))
    
    percent_diff_lst.append(percent_diff)
    if(len(percent_diff_lst)>ma):
        percent_diff_lst.pop(0)
    percent_diff_10ma = Average(percent_diff_lst)
    print('percent_diff_10ma : '+str(round(percent_diff_10ma,2)))
    
    trend = 0
    
    if(percent_diff_10ma>0 and percent_diff>threshold):
        trend = -1
        my_qty = my_qty - abs(int(qty_m_spot*(percent_diff/100)/qty_factor))
        ps_thr = threading.Thread(target=placeneworder, args=(-abs(int(qty_m_spot*(percent_diff/100)/qty_factor)),0))
        ps_thr.start()
        
        
    if(percent_diff_10ma<0 and percent_diff<-threshold):
        trend = 1
        my_qty = my_qty + abs(int(qty_m_spot*(percent_diff/100)/qty_factor))
        ps_thr = threading.Thread(target=placeneworder, args=(abs(int(qty_m_spot*(percent_diff/100)/qty_factor)),0))
        ps_thr.start()
#         placeneworder(abs(int(qty_m_spot*(percent_diff/100)/qty_factor)),0)
        
        
#     if(percent_diff_10ma>0 and percent_diff>5):
#         print('DOWN TREND')
#         trend = -1
#         my_qty = my_qty - 10
        
#     if(percent_diff_10ma<0 and percent_diff<-5):
#         print('UP TREND')    
#         trend = 1
#         my_qty = my_qty + 10
        
    print('my_qty : '+str(my_qty))
    
    log_data_lst.append({'buy_quantity':buy_quantity,
                         'sell_quantity':sell_quantity,
                         'diff':diff,
                         'mean':mean,
                         'percent':percent,
                         'percent_10ma':percent_10ma,
                         'percent_diff':percent_diff,
                         'percent_diff_10ma':percent_diff_10ma,
                         'trend':trend,
                         'my_qty':my_qty,
                         'ltp':ticks[0]['last_price'],
                         'date':ticks[0]['timestamp'],
    })
    
    print( 'LTP : ' + str(ticks[0]['last_price']))
#     print( 'tick : ' + str(ticks[0]))
    
    if(counter%5==0):
        export_calc()
    
    counter = counter + 1

def on_connect(ws, response):
    global inst_token
    ws.subscribe([inst_token])
    ws.set_mode(ws.MODE_FULL, [inst_token])

def on_close(ws, code, reason):
    ws.stop()

def on_error(ws, code, reason):
    logging.error("closed connection on error: {} {}".format(code, reason))

def on_noreconnect(ws):
    logging.error("Reconnecting the websocket failed")

def on_reconnect(ws, attempt_count):
    logging.debug("Reconnecting the websocket: {}".format(attempt_count))
    
def on_order_update(ws, data):
    print('oder update')

module.kws.on_error = on_error
module.kws.on_noreconnect = on_noreconnect
module.kws.on_reconnect = on_reconnect
module.kws.on_order_update = on_order_update
module.kws.on_ticks = on_ticks
module.kws.on_connect = on_connect
module.kws.on_close = on_close

module.kws.connect()